In [1]:
import os
from glob import glob
import pandas as pd
import sys
import openpyxl


install required packages 

In [527]:
def install(name):
    import subprocess
    subprocess.call(['pip', 'install', name])
       

In [579]:
install('openpyxl')
install('xlsxwriter')

#### define paths and change directories

In [73]:
data_dir='/Users/daelsaid/scratch/Neuropsych_organized/scratch_testing'
os.chdir(data_dir)

#### rename_cols

In [3]:
column_names=['A_equation_old','A_key_old','table_break','B_equation','B_key']

### function that sets cell value to correct equation and answer
#### if equation = incorrect, set to corr
#### if ans = incorrect, set to corr

In [4]:
data={'A_equation_old': ['16 - 4','17 - 5'],'A_key_old': ["12", "2"]}

def update_row_vals(row,data=data):
    if row == data['A_equation_old'][0]:
        row = '16 - 14'
    if row == data['A_key_old'][0]:
        row = '2'                
    if row == data['A_equation_old'][1]:
        row = '17 - 15'
    if row == data['A_key_old'][1]:
        row = '2'                
    return row        

#### list filename, subject PID and visit
#### apply function from above to each relevant row
#### check each respective row equation and answer to ensure change was done correctly


In [ ]:
for subj_data_file in os.listdir(data_dir):
    subj_df=pd.read_excel(subj_data_file,sheetname=4,header=[0],skiprows=None,dtype=str)
    subj_df.columns=column_names
    subj=subj_data_file.split('_')[2]
    visit=subj_data_file.split('.')[0].split('_')[3]
    print subj,visit
    subj_df.A_equation_old.iloc[9] = subj_df.A_equation_old.apply(update_row_vals).iloc[9]
    subj_df.A_equation_old.iloc[62] = subj_df.A_equation_old.apply(update_row_vals).iloc[62]
    subj_df.A_key_old.iloc[9] = subj_df.A_key_old.apply(update_row_vals).iloc[9]
    subj_df.A_key_old.iloc[62] = subj_df.A_key_old.apply(update_row_vals).iloc[62]
    new=subj+'test_'+visit+'.xlsx'
    test='MET_Subtraction_'+subj+'_'+visit+'_test.xlsx'
    orig='MET_Subtraction_'+subj+'_'+visit+'.xlsx'
    #     subj_df.to_excel('MET_Subtraction_'+subj+'_'+visit+'_'+'test.xlsx',index=False,sheet_name='Version_Pool',header=['A','','','B',''],na_rep='')

    

In [ ]:
all_assess_dirs='/Users/daelsaid/scratch/Neuropsych_organized/scratch_testing/all'
os.chdir(all_assess_dirs)

for subj_data_file in os.listdir(all_assess_dirs):
    print subj_data_file
    try:
        subj_df=pd.read_excel(subj_data_file,sheetname='Import_Formula',header=[0],skiprows=None,dtype=str)
        subj=subj_data_file.split('_')[2]
        visit=subj_data_file.split('.')[0].split('_')[3]
        file_prefix=subj_data_file.split('.')[0]
#         subj_df.to_excel(file_prefix+'_import.xlsx',index=False,sheet_name='Import',na_rep='')
    except:
        print 'error for %s', file_prefix
        continue
   

In [ ]:
for subj_data_file in os.listdir(data_dir):
    subj_df=pd.read_excel(subj_data_file,sheetname=4,skiprows=1,dtype=str)
    subj_df.columns=column_names
    subj=subj_data_file.split('_')[2]
    visit=subj_data_file.split('.')[0].split('_')[3]
#     print ['OLD: '+ subj_df.A_equation_old.iloc[8]+' = '+ subj_df.A_key_old.iloc[8]],subj_df.A_equation_old.apply(update_row_vals).iloc[8],subj_df.A_key_old.apply(update_row_vals).iloc[8]
    print subj,visit
#     subj_df.A_equation_old.iloc[8] = subj_df.A_equation_old.apply(update_row_vals).iloc[8]
#     subj_df.A_equation_old.iloc[61] = subj_df.A_equation_old.apply(update_row_vals).iloc[61]
#     subj_df.A_key_old.iloc[8] = subj_df.A_key_old.apply(update_row_vals).iloc[8]
#     subj_df.A_key_old.iloc[61] = subj_df.A_key_old.apply(update_row_vals).iloc[61]
# #     subj_df.to_excel('MET_Subtraction_'+subj+'_'+visit+'_'+'test.xlsx',index=False,sheet_name='Version_Pool',header=['A','','','B',''],na_rep='')
#     print subj_df.iloc[8][0],subj_df.iloc[8][1]
#     print subj_df.iloc[61][0],subj_df.iloc[61][1]
#     append_df_to_excel(orig,subj_df,'Version_Pool',startrow=None,truncate_sheet=True)

In [41]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,truncate_sheet=False, **to_excel_kwargs):
    from openpyxl import load_workbook
    import pandas as pd

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='xlsxwriter')

    try:
        FileNotFoundError
    except NameError:
        FileNotFoundError = IOError

    try:
        # try to open an existing workbook
        writer.book = load_workbook(new)
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row
        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            idx = writer.book.sheetnames.index(sheet_name)
            writer.book.remove(writer.book.worksheets[idx])
            writer.book.create_sheet(sheet_name, idx)
        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        pass

    if startrow is None:
        startrow = 0

    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)
    writer.save()

In [45]:
def append_sheet_to_master(writer,master_file_path, current_file_path, sheet_name):
    try:
        master.book = load_workbook(master_file_path)
        master_writer = pd.ExcelWriter(master_file_path, engine='openpyxl')
        master_writer.book = master.book
        master_writer.sheets = dict((ws.title, ws) for ws in master_book.worksheets)
        current_frames = pandas.ExcelFile(current_file_path).parse(pandas.ExcelFile(current_file_path).sheet_names[0],
                                                               header=None,
                                                               index_col=None)
        current_frames.to_excel(master_writer, sheet_name, index=None, header=False)
        master_writer.save()
    except Exception as e:
        raise e

In [46]:
dfs = pd.read_excel('MET_Subtraction_9470_1.xlsx', sheetname=None)
repl_dict = {'a': 'b', 'c': 'd', 'e': 'f', 'g': 'h', 'i': 'j'}
data={'A_equation_old': ['16 - 4','17 - 5'],'A_key_old': ["12", "2"]}

writer = pd.ExcelWriter('filename.xlsx')
for sheetname, df in dfs.items():
    df = df.replace(repl_dict)
    df.to_excel(writer, sheetname, index=False)
writer.save()